Imports

In [1]:
import pandas as pd
import math
import numpy as np
import seaborn
from geopy.distance import vincenty

Get data and basic filters

In [2]:
df_county_raw = pd.read_csv('data/location_population.csv', index_col='Unnamed: 0')
print len(df_county_raw['State'].unique())
df_county_raw = df_county_raw.reset_index(drop=True)
df_county_raw.head()

48


,County,State,Rank,Population,latitude,longitude
0,Autauga County,AL,907,"55,514",32.524961,-86.629505
1,Baldwin County,AL,332,"190,790",30.607402,-87.718027
2,Barbour County,AL,1523,"27,201",31.813697,-85.481079
3,Bibb County,AL,1707,"22,597",33.032728,-87.080731
4,Blount County,AL,882,"57,826",33.951944,-86.582932


Advanced Filtering

In [3]:
def to_float(x):
    x = x.replace(',', '')
    return float(x)
df_county_raw[['Population']] = df_county_raw['Population'].apply(to_float)

Slice of data, cities bigger than ...

Calculate Distances Algorithm

In [4]:
len(df_county_raw)

3107

In [5]:
def get_distance(county, df_city, rng, rng2=0):

        p1 = [county['latitude'], county['longitude']]

        # calculate region to search for big cities

        out_square_lat = [county['latitude'] + rng, county['latitude'] - rng]
        out_square_lon = [county['longitude'] + rng, county['longitude'] - rng]

        in_square_lat = [county['latitude'] + rng2, county['latitude'] - rng2]
        in_square_lon = [county['longitude'] + rng2, county['longitude'] - rng2]

        j = 0
        s_dist = 100000
        for k, city in df_city.iterrows():
            p2 = [city['latitude'], city['longitude']]

            if (out_square_lat[1] < p2[0] < out_square_lat[0]) & (out_square_lon[1] < p2[1] < out_square_lon[0]):
                if not ((in_square_lat[1] < p2[0] < in_square_lat[0]) & (in_square_lon[1] < p2[1] < in_square_lon[0])):
                    j += 1

                dist = vincenty(p1, p2).km

                if dist < s_dist:
                    s_dist = dist

        return s_dist, j

In [11]:
df_county = df_county_raw
#cities_bigger_than = [100000]
cities_bigger_than = [300000, 500000, 1000000]
search_range = [5, 10, 15, 20, 30, 50, 100, 1000]
for bigger_than in cities_bigger_than:
    print 'bigger', bigger_than
    
    df_city = df_county_raw[df_county_raw['Population'] >= bigger_than] #get counties bigger than
    df_county['distance_{}'.format(bigger_than)] = np.nan #create empty column
    #print df_city
    for i, county in df_county.iterrows():
        
        if not county['Population'] >= bigger_than:

            for k in range(len(search_range)):
                if k == 0:
                    smaller_distance, j = get_distance(county, df_city, search_range[k])
                else:
                    print search_range[k]
                    smaller_distance, j = get_distance(county, df_city, search_range[k], search_range[k-1])

                if j:
                    break
        else:
            smaller_distance = 0
        if i % 100 == 0:
            print i
        try:
            df_county['distance_{}'.format(bigger_than)].iloc[i] = smaller_distance
        except Exception as inst:
            print inst
            
df_county.head()

bigger 300000
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1600
10
10
10
10
10
10
10
10
10
10
1700
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1800
1900
2000
2100
2200
2300
10
10
2400
2500
2600
10
2700
2800
2900
3000
3100
bigger 500000
0
100
200
300
400
500
600
10
10
10
10
10
10
10
10
10
10
700
800
900
1000
1100
1200
1300
1400
1500
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1600
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1700
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1800
1900
2000
2100
2200
2300
10
10
10
10
2400
2500
2600
10
2700
2800
2900
3000
3100
bigger 1000000
0
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
100
200
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


,County,State,Rank,Population,latitude,longitude,distance_100000,distance_300000,distance_500000,distance_1000000
0,Autauga County,AL,907,55514.0,32.524961,-86.629505,55.468394,120.387228,120.387228,656.514912
1,Baldwin County,AL,332,190790.0,30.607402,-87.718027,0.000000,36.620510,340.608832,601.274988
2,Barbour County,AL,1523,27201.0,31.813697,-85.481079,74.924444,216.820215,234.120887,526.289453
3,Bibb County,AL,1707,22597.0,33.032728,-87.080731,45.862509,65.199386,65.199386,681.752585
4,Blount County,AL,882,57826.0,33.951944,-86.582932,40.717787,47.727390,47.727390,612.043635


In [12]:
df_county.to_csv('data/county_distances.csv', index=False)